In [109]:
import pandas as pnd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

Cargamos el csv para empezar el trabajo

In [110]:
mensajesTwitter = pnd.read_csv("datas/calentamientoClimatico_sin_preparar.csv", delimiter=";")
mensajesTwitter.head(10)

,Column1,Column2,Column3
0,tweet,existence,existence.confidence
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
6,RT @sejorg: RT @JaymiHeimbuch: Ocean Saltiness...,Yes,1
7,Global warming evidence all around us|A messag...,Yes,1
8,Migratory Birds' New Climate Change Strategy: ...,Yes,1
9,Southern Africa: Competing for Limpopo Water: ...,Yes,1


Vemos la dimensión del dataset

In [111]:
mensajesTwitter.shape

(6091, 3)

Eliminamos la primera fila que no tiene ninguan utilidad

In [112]:
mensajesTwitter = mensajesTwitter.drop([0],axis=0)

Cambiamos el nombre de la columnas para reconocer cada una de ellas

In [113]:
column_names=['TWEET', 'CREENCIA', 'CONFIANZA']
mensajesTwitter.columns = column_names
mensajesTwitter

,TWEET,CREENCIA,CONFIANZA
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
...,...,...,...
6086,"@bloodless_coup ""The phrase 'global warming' s...",Yes,1
6087,Virginia to Investigate Global Warming Scienti...,NaN,1
6088,Global warming you tube parody you will enjoy ...,No,0.6411
6089,One-Eyed Golfer: Don't dare tell me about glob...,No,1


Miramos a ver si tenemos algun valor nulo

In [114]:
mensajesTwitter.isna().sum()

TWEET           0
CREENCIA     1865
CONFIANZA       3
dtype: int64

Eliminamos las filas con valores nulos

In [115]:
mensajesTwitter = mensajesTwitter.dropna()
mensajesTwitter

,TWEET,CREENCIA,CONFIANZA
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
...,...,...,...
6085,It's 83�_� and climbing in NYC. August weather...,Yes,1
6086,"@bloodless_coup ""The phrase 'global warming' s...",Yes,1
6088,Global warming you tube parody you will enjoy ...,No,0.6411
6089,One-Eyed Golfer: Don't dare tell me about glob...,No,1


Verificamos, una vez realizada la limpieza de nulos, que ya no queda ninguno

In [116]:
mensajesTwitter.isna().sum()

TWEET        0
CREENCIA     0
CONFIANZA    0
dtype: int64

Vemos los valores únicos que tienen las columnas

In [117]:
mensajesTwitter['CREENCIA'].unique()

array(['Yes', 'No'], dtype=object)

In [119]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Categorizamos la columna CREENCIA 

In [129]:
mensajesTwitter['CREENCIA'] = (mensajesTwitter['CREENCIA']=='Yes').astype(int)
print(mensajesTwitter.head(100))

                                                 TWEET  CREENCIA CONFIANZA
1    global warm report urg govern act brussel belg...         0         1
2                fight poverti global warm africa link         0         1
3    carbon offset vatican forest fail reduc global...         0    0.8786
4    carbon offset vatican forest fail reduc global...         0         1
5           uruguay tool need vulner climat chang link         0    0.8087
..                                                 ...       ...       ...
135          report save whale save u global warm link         0     0.823
137  arctic beauti black white alaska effect global...         0         1
138  epa report document real impact climat chang 2...         0         1
139  canadian ceo keen green global counterpart pre...         0    0.7896
140  rt user ask g8 g2 support biochar fight climat...         0     0.617

[100 rows x 3 columns]


<ipython-input-129-28bb6459ddde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['CREENCIA'] = (mensajesTwitter['CREENCIA']=='Yes').astype(int)


Función de normalización

In [130]:
import re
def normalizacion(mensaje):
    mensaje = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', mensaje)
    mensaje = re.sub('@[^\s]+','USER', mensaje)
    mensaje = mensaje.lower().replace("ё", "е")
    mensaje = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', mensaje)
    mensaje = re.sub(' +',' ', mensaje)
    return mensaje.strip()

Normalizamos

In [131]:
mensajesTwitter["TWEET"] = mensajesTwitter["TWEET"].apply(normalizacion)
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warm report urg govern act brussel belg...         0         1
2               fight poverti global warm africa link         0         1
3   carbon offset vatican forest fail reduc global...         0    0.8786
4   carbon offset vatican forest fail reduc global...         0         1
5          uruguay tool need vulner climat chang link         0    0.8087
6   rt user rt user ocean salti show global warm i...         0         1
7   global warm evid around u messag global warm d...         0         1
8   migratori bird new climat chang strategi stay ...         0         1
9   southern africa compet limpopo water climat ch...         0         1
10  global warm impact wheat rice product india lu...         0         1


<ipython-input-131-d931aa831d20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter["TWEET"] = mensajesTwitter["TWEET"].apply(normalizacion)


Carga de StopWords

In [132]:
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

Eliminación de las Stops Words en las distintas frases

In [133]:
mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([palabra for palabra in mensaje.split() if palabra not in (stopWords)]))
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warm report urg govern act brussel belg...         0         1
2               fight poverti global warm africa link         0         1
3   carbon offset vatican forest fail reduc global...         0    0.8786
4   carbon offset vatican forest fail reduc global...         0         1
5          uruguay tool need vulner climat chang link         0    0.8087
6   rt user rt user ocean salti show global warm i...         0         1
7   global warm evid around u messag global warm d...         0         1
8   migratori bird new climat chang strategi stay ...         0         1
9   southern africa compet limpopo water climat ch...         0         1
10  global warm impact wheat rice product india lu...         0         1


<ipython-input-133-57bbf70baf03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([palabra for palabra in mensaje.split() if palabra not in (stopWords)]))


Aplicación de stemming

In [134]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([stemmer.stem(palabra) for palabra in mensaje.split(' ')]))
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warm report urg govern act brussel belg...         0         1
2               fight poverti global warm africa link         0         1
3   carbon offset vatican forest fail reduc global...         0    0.8786
4   carbon offset vatican forest fail reduc global...         0         1
5          uruguay tool need vulner climat chang link         0    0.8087
6   rt user rt user ocean salti show global warm i...         0         1
7   global warm evid around u messag global warm d...         0         1
8   migratori bird new climat chang strategi stay ...         0         1
9   southern africa compet limpopo water climat ch...         0         1
10  global warm impact wheat rice product india lu...         0         1


<ipython-input-134-6bad69e7505d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([stemmer.stem(palabra) for palabra in mensaje.split(' ')]))


Lematización

In [135]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([lemmatizer.lemmatize(palabra) for palabra in mensaje.split(' ')]))
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warm report urg govern act brussel belg...         0         1
2               fight poverti global warm africa link         0         1
3   carbon offset vatican forest fail reduc global...         0    0.8786
4   carbon offset vatican forest fail reduc global...         0         1
5          uruguay tool need vulner climat chang link         0    0.8087
6   rt user rt user ocean salti show global warm i...         0         1
7   global warm evid around u messag global warm d...         0         1
8   migratori bird new climat chang strategi stay ...         0         1
9   southern africa compet limpopo water climat ch...         0         1
10  global warm impact wheat rice product india lu...         0         1


<ipython-input-135-0510f4d08012>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([lemmatizer.lemmatize(palabra) for palabra in mensaje.split(' ')]))


Conjunto de aprendizaje y de prueba:

In [136]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mensajesTwitter['TWEET'].values,  mensajesTwitter['CREENCIA'].values,test_size=0.2)

In [137]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

etapas_aprendizaje = Pipeline([('frequencia', CountVectorizer()),
                                  ('tfidf', TfidfTransformer()),
                                  ('algoritmo', MultinomialNB())])

Aprendizaje

In [138]:
modelo = etapas_aprendizaje.fit(X_train,y_train)

In [139]:
from sklearn.metrics import classification_report
print(classification_report(y_test, modelo.predict(X_test), digits=4))

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       845

    accuracy                         1.0000       845
   macro avg     1.0000    1.0000    1.0000       845
weighted avg     1.0000    1.0000    1.0000       845



In [140]:
frase = "Why should trust scientists with global warming if they didnt know Pluto wasnt a planet"
frase

'Why should trust scientists with global warming if they didnt know Pluto wasnt a planet'

In [141]:
#Normalización
frase = normalizacion(frase)

In [142]:
#Eliminación de las stops words
frase = ' '.join([palabra for palabra in frase.split() if palabra not in (stopWords)])

In [143]:
#Aplicación de stemming
frase =  ' '.join([stemmer.stem(palabra) for palabra in frase.split(' ')])

In [144]:
#Lematización
frase = ' '.join([lemmatizer.lemmatize(palabra) for palabra in frase.split(' ')])
print (frase)

trust scientist global warm didnt know pluto wasnt planet


In [145]:
prediccion = modelo.predict([frase])
print(prediccion)
if(prediccion[0]==0):
    print(">> No cree en el calentamiento climático...")
else:
    print(">> Cree en el calentamiento climático...")

[0]
>> No cree en el calentamiento climático...
